In [ ]:
import streamlit as st
import pandas as pd
from PIL import Image
import numpy as np 
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py


df = pd.read_csv("Food.csv")
df = pd.read_csv("Environnement.csv", encoding='latin-1')# csv file is encoding as latin-1 type
df_countrycode=pd.read_csv('stat.csv')
df.drop(['Area Code','Months Code','Element Code','Element','Unit'],axis=1,inplace=True)
df.dropna()
df.rename(columns = {'Area':'Pays'},inplace = True)
df.rename(columns = {'Months':'Mois'},inplace = True)
df.set_index('Mois', inplace=True)
df.rename({'Dec\x96Jan\x96Feb': 'Winter', 'Mar\x96Apr\x96May': 'Spring', 'Jun\x96Jul\x96Aug':'Summer','Sep\x96Oct\x96Nov':'Fall'}, axis='index',inplace = True)
df.reset_index(inplace = True)
df_countrycode.rename(columns = {'Country':'Pays','ISO3 Code':'Country Code'},inplace=True)
df = pd.merge(df, df_countrycode, how='outer', on='Pays')
df = df.melt(id_vars=["Pays","Mois",], var_name="year", value_name="tem_change")
df["year"] = [i.split("Y")[-1] for i in df.year]

st.title("oui")

st.markdown("oui")


df_tem_change = df.copy() # do not lose 'df', I made copy of it
df_tem_change = df_tem_change[df_tem_change['Mois'] == 'Meteorological year'] # chose just year data
df_tem_change.drop(['Mois'],axis=1,inplace=True) # dropped Months column
df_tem_change.to_csv('Temperature_change_Data.csv',index=False) # export data to share with the project group members
df_map = df.copy() # do not lose 'df', I made copy of it
df_map = df_map[df_map['Mois'] == 'Meteorological year'] # chose yearly base data


df_map['°C'] = ['<=-1.5' if x<=(-1.5) else '<=-1.0' if (-1.5)<x<=(-1.0) else '<=0.0' if (-1.0)<x<=0.0  else '<=0.5' if 0.0<x<=0.5 else '<=1.5' if 0.5<x<=1.5 else '>1.5' if 1.5<=x<10 else 'None' for x in df_map['tem_change']]

fig = px.choropleth(df_map, locations="Country Code", # used plotly express choropleth for animation plot
                    color="°C",
                    locationmode='ISO-3',
                    hover_name="Pays",
                    hover_data=['tem_change'],
                    animation_frame =df_map.year,
                    labels={'tem_change':'The Temperature Change', '°C':'°C'},
                    category_orders={'°C':['<=-1.5','<=-1.0','<=0.0','<=0.5','<=1.5','>1.5','None']},
                    color_discrete_map={'<=-1.5':"#08519c",'<=-1.0':"#9ecae1",'<=0.0':"#eff3ff",'<=0.5':"#ffffb2",'<=1.5': "#fd8d3c",'>1.5':"#bd0026",'None':"#252525"},
                    title = 'Temperature Change - 1961 - 2019')









df_c =df.copy()
df_c.set_index("year", inplace=True)
df_c = df_c.loc[['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']]
df_c.reset_index(inplace = True)


df_c = df_c.groupby(
   ['Pays',]
).agg(
    {
        'tem_change':'mean', 
        
    }
)
df_c.reset_index(inplace = True)
df_c = df_c.sort_values(by=['tem_change'],ascending=True).head(10)

fig = px.bar(df_c, x="Pays", y='tem_change',text='tem_change' , title="Top ten countries that have lowest temperature change in the last decades")
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')













df_c =df.copy()
df_c.set_index("year", inplace=True)
df_c = df_c.loc[['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']]
df_c.reset_index(inplace = True)


df_c = df_c.groupby(
   ['Pays',]
).agg(
    {
        'tem_change':'mean', 
        
    }
)
df_c.reset_index(inplace = True)
df_c = df_c.sort_values(by=['tem_change'],ascending=False).head(10)

fig = px.bar(df_c, x="Pays", y='tem_change' ,text='tem_change', title="Top ten countries that have highest temperature change in the last decades")










food_df= df.groupby("Food product")['Total_emissions'].sum().reset_index()
trace = go.Scatter(
    y = food_df.Total_emissions,
    x = food_df["Food product"],
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = food_df.Total_emissions*2,
        color = food_df.Total_emissions,
        colorscale='Picnic',
        showscale=True
    )
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Total Emissions par alimentation',
    hovermode= 'closest',
     xaxis= dict(
         ticklen= 5,
         showgrid=False,
        zeroline=False,
        showline=False
     ),
    yaxis=dict(
        title= 'Total Emissions',
        showgrid=False,
        zeroline=False,
        ticklen= 7,
        gridwidth= 3
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatterplot')







emission_df= df.dropna().sort_values(by= 'Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal)', ascending= True)[['Food product',
       'Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal)']]

fig, ax = plt.subplots(figsize=(10,20))
sns.set()
emission_df.set_index('Food product').plot(kind='barh', stacked=True, ax= ax, color= "lightblue")
plt.xlabel("Greenhouse gas emission per 1000kcal")
plt.title("Greenhouse Gas Emission Per 1000kcal\n", size= 25)